In [52]:
import numpy as np
import scipy.sparse as sp

from scipy.sparse.linalg import eigsh
from typing import List

## 0. tl;dr

Complete the functions
* `construct_laplacian` (4pt)
* `spectral_embedding` (2pt)
* `compute_ratio_cut`(2pt)
* `compute_normalized_cut` (2pt)

in `clustering.py`, then push the file to the remote Artemis respository.

For `spectral_embedding`, please use the provided function `deterministic_eigsh` and not `scipy.sparse.linalg.eigsh`.

# Programming Task 4: Spectral clustering

The goal of this task is to find groups of users with similar preferences using **Spectral clustering**. 
You are given a fragment of the Yelp social network, represented by an undirected weighted graph.
Nodes in the graph represent users.
If two users are connected by an edge of weight $w$, it means that they have both left positive reviews to the same $w$ restaurants.

Additionally, you are given a matrix `F` that encodes user preferences to different categories of restaurants. If `F[i, c] = 1`, then user `i` likes restaurants in category `c`.

## General remarks
Where possible, please use sparse matrix operations (see [Scipy Sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)). Otherwise, we cannot guarantee that your code will run on the test machines.

Do not add or modify any code outside of the following comment blocks, or where otherwise explicitly stated.

``` python
##########################################################
# YOUR CODE HERE
...
##########################################################
```

The following things are **NOT** allowed:
- Using additional `import` statements **and submodules of `sp` are not allowed such as `sp.csgraph.laplacian`**
- Copying / reusing code from other sources (e.g. code by other students)
- Posting your code in public threads on Piazza

## Load the data

* `N` = number of users (nodes in the graph)
* `C` = number of categories
* The graph is stored as a _sparse adjacency matrix_ `A` (shape `[N, N]`).
* User preferences are stored in a _feature matrix_ `F` (shape `[N, C]`). They will only be used for the final part of the assignment (Part 3)
* Name of each category is provided in the list `categories` (length `[C]`).

In [53]:
A = sp.load_npz('A.npz')
F = np.load('F.npy')
categories = np.load('categories.npy', allow_pickle=True).tolist()

In [54]:
assert A.shape[0] == F.shape[0]
assert F.shape[1] == len(categories)

In [4]:
print(f'The adjacency matrix is {"symmetric" if (A != A.T).sum() == 0 else "asymmetric"}')

The adjacency matrix is symmetric


# 1. Spectral clustering (6pt)
## 1.1. Constructing the graph Laplacian (4pt)

First, we need functionality to construct the Laplacian for the given graph.

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

If our goal is to minimize the **ratio cut**, we use the **unnormalized Laplacian**, defined as
$$L_{unnorm} = D - A.$$

If our goal is to minimze the **normalized cut**, we use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

**Before proceeding, implement the following function** from ``clustering.py``:
* `construct_laplacian` (4pt)

In [5]:
from clustering import construct_laplacian

## 1.2. Spectral embedding (2 pt)

Next, we want to use the implemented functionality to compute spectral embeddings.

This spectral embedding is given by the $k$ smallest eigenvalues of:
* the unnormalized Laplacian, if we want to minimize the ratio cut,
* the normalized Laplacian, if we want to minimize the normalized cut.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).
The function `eigsh` directly allows you to find the smallest / largest eigenvalues by specifying the `k` and `which` parameters. 

Keep in mind that the Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [6]:
help(eigsh)

Help on function eigsh in module scipy.sparse.linalg.eigen.arpack.arpack:

eigsh(A, k=6, M=None, sigma=None, which='LM', v0=None, ncv=None, maxiter=None, tol=0, return_eigenvectors=True, Minv=None, OPinv=None, mode='normal')
    Find k eigenvalues and eigenvectors of the real symmetric square matrix
    or complex Hermitian matrix A.
    
    Solves ``A * x[i] = w[i] * x[i]``, the standard eigenvalue problem for
    w[i] eigenvalues with corresponding eigenvectors x[i].
    
    If M is specified, solves ``A * x[i] = w[i] * M * x[i]``, the
    generalized eigenvalue problem for w[i] eigenvalues
    with corresponding eigenvectors x[i].
    
    Note that there is no specialized routine for the case when A is a complex
    Hermitian matrix. In this case, ``eigsh()`` will call ``eigs()`` and return the
    real parts of the eigenvalues thus obtained.
    
    Parameters
    ----------
    A : ndarray, sparse matrix or LinearOperator
        A square operator representing the operation ``

**Before proceeding, implement the following function** from ``clustering.py``:
* `construct_laplacian` (2pt)

In [7]:
from clustering import spectral_embedding

## 1.3. Determining the clusters based on the spectral embedding

Once we have the spectral embeddings, we can use k-means clustering to assign nodes to clusters.

Note that when using the **normalized Laplacian**, the rows of the embedding matrix **have to** be normalized to have unit $L_2$ norm. The justification of this approach is not trivial. If you are interested, the paper "A tutorial on spectral clustering" by Ulrike von Luxburg. Section 7 covers a justification involving Perturbation Theory. 

In [8]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [9]:
def spectral_clustering(A: sp.csr_matrix, num_clusters: int, norm_laplacian: bool, seed: int = 42) -> np.array:
    """Perform spectral clustering on the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
    seed : int, default 42
        Random seed to use for the `KMeans` clustering.
        
    Returns
    -------
    z_pred : np.array, shape [N]
        Predicted cluster indicators for each node.
        
    """
    model = KMeans(num_clusters, random_state=seed)

    embedding = spectral_embedding(A, num_clusters, norm_laplacian)
    if norm_laplacian == True:
        embedding = normalize(embedding, norm='l2', axis=1)
    z_pred = model.fit_predict(embedding)

    return z_pred

# 2. Quantitative evaluation (4 pt)

Your next task is to implement functions for computing the **ratio cut** and **normalized cut** for a given partition.

Ratio cut and normalized cut are defined on page 14 of the lecture slides.  
The expression $vol(C_i)$ refers to the number of edges incident to nodes from cluster $C_i$.

The function `labels_to_list_of_clusters` in `clustering.py` can be helpful here.

**Before proceeding, implement the following function** from ``clustering.py``:
* `compute_ratio_cut` (2pt)
* `compute_normalized_cut` (2pt)

In [10]:
from clustering import compute_ratio_cut, compute_normalized_cut, labels_to_list_of_clusters

Notice, how using the unnormalized Laplacian leads to a much better ratio cut, while the normalized Laplacian leads to better normalized cut.

In [11]:
num_clusters = 6

In [12]:
np.random.seed(12903)
norm_laplacian = False
z_unnorm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_unnorm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_unnorm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_unnorm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_unnorm)]))

When using L_unnorm:
 ratio cut = 369.109
 normalized cut = 5.000
 sizes of partitions are: [3379, 1, 1, 1, 1, 1]


In [13]:
np.random.seed(12323)
norm_laplacian = True
z_norm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_norm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_norm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_norm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_norm)]))

When using L_norm:
 ratio cut = 5942.851
 normalized cut = 4.343
 sizes of partitions are: [742, 577, 754, 572, 350, 389]


# 3. Visualizing the results

Finally, we can use our clustering methods to determine the most popular restaurant categories in each cluster and how many positive ratings were given by users of that cluster.

In [14]:
def print_top_categories_for_each_cluster(top_k: int, z: np.array, F: sp.csr_matrix, categories: List[str]):
    """Print the top-K categories among users in each cluster.
    For each cluster, the function prints names of the top-K categories,
    and number of users that like the respective category (separated by a comma).
    The function doesn't return anything, just prints the output.
    
    Parameters
    ----------
    top_k : int
        Number of most popular categories to print for each cluster.
    z : np.array, shape [N]
        Cluster labels.
    F : sp.csr_matrix, shape [N, C]
        Matrix that tells preferences of each user to each category.
        F[i, c] = 1 if user i gave at least one positive review to at least one restaurant in category c.
    categories : list, shape [C]
        Names of the categories.
        
    """

    list_of_clusters = labels_to_list_of_clusters(z)
    
    for ix, clust in enumerate(list_of_clusters):
        cat_ratings = sum(F[i] for i in clust)
        top_cats = np.argsort(-cat_ratings)[:top_k]
        print('Most popular categories in cluster {}'.format(ix))
        for t in top_cats:
            print(f' - {categories[t]}, {int(cat_ratings[t])}')
        print()

In [15]:
np.random.seed(23142)
z_norm = spectral_clustering(A, num_clusters, True)
r = print_top_categories_for_each_cluster(5, z_norm, F, categories)

Most popular categories in cluster 0
 - Breakfast & Brunch, 636
 - Sandwiches, 528
 - Italian, 514
 - Pizza, 482
 - Coffee & Tea, 473

Most popular categories in cluster 1
 - Japanese, 529
 - Chinese, 441
 - Asian Fusion, 414
 - Sushi Bars, 408
 - Korean, 406

Most popular categories in cluster 2
 - Breakfast & Brunch, 664
 - Italian, 626
 - American (Traditional), 518
 - Sandwiches, 518
 - Pizza, 485

Most popular categories in cluster 3
 - Japanese, 507
 - Breakfast & Brunch, 462
 - Sandwiches, 435
 - Italian, 417
 - Asian Fusion, 414

Most popular categories in cluster 4
 - Seafood, 315
 - Mexican, 314
 - Sandwiches, 294
 - Japanese, 291
 - Breakfast & Brunch, 286

Most popular categories in cluster 5
 - Specialty Food, 356
 - Thai, 355
 - Breakfast & Brunch, 348
 - Japanese, 333
 - Ethnic Food, 330

